In [30]:
in_height, in_width = 3,3
filt_height, filt_width = 2,2
n_chan = 3
out_height = (in_height-filt_height+1)
out_width = (in_width-filt_width+1)

In [31]:
data = [1,2,3,4,5,6,7,8,9,
        9,8,7,6,5,4,3,2,1,
        2,4,6,8,10,12,14,16,18]

depthwise_res = [0] * (out_height * out_width * n_chan)

depthwise_weights = [1,0,0,1] * n_chan

depthwise_biases = [0] * n_chan

In [32]:
def depthwise_conv(data, depthwise_res, depthwise_weights, depthwise_biases):
    for c in range(n_chan):
        channel_start = c * in_height * in_width
        kernel_start = c * filt_height * filt_width

        for h in range(out_height):
            for w in range(out_width):
                _sum = depthwise_biases[c]
                for i in range(filt_height):
                    for j in range(filt_width):
                        data_idx = channel_start + (h + i) * in_width + (w + j)
                        kernel_idx = kernel_start + i * filt_width + j
                        _sum += data[data_idx] * depthwise_weights[kernel_idx]

                res_idx = c * (out_height * out_width) + h * out_width + w
                depthwise_res[res_idx] = _sum

    return depthwise_res

# Redefine variables and rerun the function\
depthwise_res = depthwise_conv(data, depthwise_res, depthwise_weights, depthwise_biases)
depthwise_res

[6, 8, 12, 14, 14, 12, 8, 6, 12, 16, 24, 28]

In [33]:
n_filt = 2
res = [0] * (out_height * out_width * n_filt)
pointwise_weights = [1,0,1,0,1,1] * n_chan
pointwise_biases = [0] * n_filt

In [34]:
def pointwise_conv_2d_latency_cl(depthwise_res, res, pointwise_weights, pointwise_biases):
    for h in range(out_height):
        for w in range(out_width):
            for f in range(n_filt):
                _sum = pointwise_biases[f]  
                for c in range(n_chan):
                    idx = c * (out_height * out_width) + h * out_width + w
                    weight_idx = f * n_chan + c  
                    _sum += depthwise_res[idx] * pointwise_weights[weight_idx]
                
                res[f * (out_height * out_width) + h * out_width + w] = _sum
    return res

res = pointwise_conv_2d_latency_cl(depthwise_res, res, pointwise_weights, pointwise_biases)
res

[18, 24, 36, 42, 26, 28, 32, 34]